In [91]:
import pickle
import pandas as pd
import numpy as np
import sys
from IPython.display import display, clear_output
from datetime import datetime
from tqdm import tqdm_notebook
import itertools
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
import os

Филигранно распикливаем ([с возможной утечкой памяти:)](https://stackoverflow.com/questions/7395542/is-explicitly-closing-files-important))

In [2]:
players_dict = pickle.load(open('data/chgk/players.pkl', 'rb'))
results_dict = pickle.load(open('data/chgk/results.pkl', 'rb'))
tournaments_dict = pickle.load(open('data/chgk/tournaments.pkl', 'rb'))

In [3]:
tournaments_dict_train = {k: v for k, v in tournaments_dict.items() if v['dateStart'].startswith('2019')}
tournaments_dict_test = {k: v for k, v in tournaments_dict.items() if v['dateStart'].startswith('2020')}

`players` и `tournaments` сразу в датафрейм, `results` пока что в словарь датафреймов 

In [4]:
results_dict_train = {k: v for k, v in results_dict.items() if k in tournaments_dict_train.keys()}
results_dict_test = {k: v for k, v in results_dict.items() if k in tournaments_dict_test.keys()}

Фильтруем `results`, оставляем только те, где есть `mask`

In [5]:
results_filtered_train = {}
results_filtered_test = {}
for k, res in results_dict.items():
    t_res_new = []
    for t_res in results_dict[k]:
        if 'mask' in t_res.keys():
            if t_res['mask'] is not None:
                if 'X' not in t_res['mask'] and '?' not in t_res['mask']:
                    t_res_new.append(t_res)
    if t_res_new:
        if tournaments_dict[k]['dateStart'].startswith('2019'):
            results_filtered_train[k] = t_res_new
        if tournaments_dict[k]['dateStart'].startswith('2020'):
            results_filtered_test[k] = t_res_new

In [8]:
tournament_question_count = {}

for k, v in results_filtered_train.items():
    tournament_question_count[k]= max([len(t_res['mask']) for t_res in v])
    
for k, v in results_filtered_test.items():
    tournament_question_count[k]= max([len(t_res['mask']) for t_res in v])

In [62]:
results_filtered_train_len = {}
results_filtered_test_len = {}
for k, v in results_filtered_train.items():
    t_res_ = []
    for t_res in v:
        if len(t_res['mask']) == tournament_question_count[k]:
            t_res_.append(t_res)
    results_filtered_train_len[k] = t_res_

for k, v in results_filtered_test.items():
    t_res_ = []
    for t_res in v:
        if len(t_res['mask']) == tournament_question_count[k]:
            t_res_.append(t_res)
    results_filtered_test_len[k] = t_res_

In [64]:
pq_df = pd.DataFrame(columns=['pid', 'qid', 'res'])

In [70]:
pid = []
qid = []
res = []
for k, v in results_filtered_train_len.items():
    for t_res in v:
        members = [m['player']['id'] for m in t_res['teamMembers']]
        pid.extend(list(itertools.chain.from_iterable(itertools.repeat(m, tournament_question_count[k]) for m in members)))
        qid.extend([f'{k}_{i}' for i in range(tournament_question_count[k])] * len(members))
        res.extend(list(map(int, t_res['mask'])) * len(members))

In [117]:
pq_df['pid'] = np.int32(pid)
pq_df['qid'] = qid
pq_df['res'] = np.int8(res)

In [118]:
pq_df['pid'].unique()

array([  6212,  18332,  18036, ..., 219555, 208933,  13126])

In [119]:
np.int8

numpy.int8

In [120]:
encoder = OneHotEncoder(categorical_features=[0, 1], sparse=True, dtype=np.int8)

In [121]:
encoder.fit(pq_df)

/Users/a18570724/.local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


OneHotEncoder(categorical_features=[0, 1], categories=None, drop=None,
              dtype=<class 'numpy.int8'>, handle_unknown='error', n_values=None,
              sparse=True)

In [122]:
pq_df_oh = encoder.transform(pq_df[['pid', 'qid']])

In [123]:
clf = LogisticRegression(solver='lbfgs', n_jobs=10)

In [124]:
clf.fit(pq_df_oh, pq_df['res'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=10, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [140]:
feature_names = np.array([fname.replace('.0','') for fname in encoder.get_feature_names(pq_df.columns[:2])])

In [144]:
players = feature_names[[fname.startswith('pid') for fname in feature_names]]
players = np.array(list(map(lambda x: np.int32(x.replace('pid_','')), players)))

In [160]:
clf.coef_[0][:len(players)].shape

(55623,)

In [158]:
players.shape

(55623,)

In [161]:
rating = pd.DataFrame({'player_id': players, 'score': clf.coef_[0][:len(players)]})

In [166]:
rating = rating.sort_values(by='score', ascending=False)

In [169]:
players_df = pd.DataFrame.from_dict(players_dict, orient='index')

In [185]:
pq_df[pq_df['pid']==27403]['res'].sum() / pq_df[pq_df['pid']==27403]['res'].shape[0]

0.7856347438752784

In [232]:
rating_named = rating.merge(players_df.rename(columns={'id':'player_id'}), on='player_id')

In [235]:
rating_named[(rating_named['name']=='Анастасия') & (rating_named['surname']=='Шутова')]

,player_id,score,place,name,patronymic,surname
224,36754,2.459624,224,Анастасия,Викторовна,Шутова


In [189]:
test_rating = {}
for tournament, result in results_filtered_test_len.items():
    for t_res in result:
        for member in t_res['teamMembers']:
            test_rating[member['player']['id']] = member['rating']

In [269]:
rating_full

,player_id,score_test,place_test,score_pred,place_pred
0,30152,14741,0,3.421566,2
1,27822,14665,1,3.298092,4
2,28751,14665,2,3.282823,5
3,30270,14665,3,3.171842,10
4,27403,14434,4,3.705755,0
...,...,...,...,...,...
22168,208771,0,28106,0.044445,22708
22169,208772,0,28107,-0.039095,25549
22170,208773,0,28108,-0.039095,25548
22171,208774,0,28109,0.044445,22709


In [270]:
rating_test = pd.DataFrame.from_dict({'player_id': list(test_rating.keys()), 'score' :list(test_rating.values())}).sort_values(by='score', ascending=False)

In [271]:
rating_test['place'] = rating_test.reset_index().index

In [272]:
rating_full = rating_test.merge(rating, on='player_id', suffixes=['_test', '_pred'])

In [274]:
rating_full.corr(method='spearman')['place_test']['place_pred']

0.6942986116547657

In [306]:
tid = []
res = []
qid = []
for k, v in results_filtered_train_len.items():
    for t_res in v:
        team_id = t_res['team']['id']
        members = [m['player']['id'] for m in t_res['teamMembers']]
        tid.extend([team_id] * len(t_res['mask']))
        qid.extend([f'{k}_{i}' for i in range(tournament_question_count[k])])
        res.extend(list(map(int, t_res['mask'])))

In [310]:
pq_df['res']

0           1
1           1
2           1
3           1
4           1
           ..
14772064    0
14772065    1
14772066    0
14772067    0
14772068    0
Name: res, Length: 14772069, dtype: int8

In [307]:
tq_df = pd.DataFrame({'tid': tid, 'qid': qid, 'res': res})

In [308]:
tq_df

,tid,qid,res
0,45556,4772_0,1
1,45556,4772_1,1
2,45556,4772_2,1
3,45556,4772_3,1
4,45556,4772_4,1
...,...,...,...
2773928,66063,6255_175,0
2773929,66063,6255_176,1
2773930,66063,6255_177,0
2773931,66063,6255_178,0


In [309]:
weights = np.random.rand()

def get_expectations(pq_df, tq_df):
    for _, (tid, qid, res) in tq_df.iterrows():
        if res == 0:
            

,pid,qid,res
0,6212,4772_0,1
1,6212,4772_1,1
2,6212,4772_2,1
3,6212,4772_3,1
4,6212,4772_4,1
...,...,...,...
14772064,215546,6255_175,0
14772065,215546,6255_176,1
14772066,215546,6255_177,0
14772067,215546,6255_178,0


In [277]:
results_filtered_train_len[4772]

[{'team': {'id': 45556,
   'name': 'Рабочее название',
   'town': {'id': 285, 'name': 'Санкт-Петербург'}},
  'mask': '111111111011111110111111111100010010',
  'current': {'name': 'Рабочее название',
   'town': {'id': 285, 'name': 'Санкт-Петербург'}},
  'questionsTotal': 28,
  'synchRequest': {'id': 56392,
   'venue': {'id': 3030, 'name': 'Санкт-Петербург'}},
  'position': 1,
  'controversials': [{'id': 91169,
    'questionNumber': 15,
    'answer': 'Мьёльнир',
    'issuedAt': '2019-01-06T13:28:48+03:00',
    'status': 'A',
    'comment': '',
    'resolvedAt': '2019-01-06T15:25:54+03:00',
    'appealJuryComment': None}],
  'flags': [],
  'teamMembers': [{'flag': 'Б',
    'usedRating': 13507,
    'rating': 13507,
    'player': {'id': 6212,
     'name': 'Юрий',
     'patronymic': 'Яковлевич',
     'surname': 'Выменец'}},
   {'flag': 'Б',
    'usedRating': 10988,
    'rating': 13185,
    'player': {'id': 18332,
     'name': 'Александр',
     'patronymic': 'Витальевич',
     'surname': 'Либ

In [239]:
torch.sparse_coo_tensor(pq_df_oh)

TypeError: sparse_coo_tensor(): argument 'size' (position 1) must be tuple of ints, not csr_matrix

In [247]:
pq_df_coo = pq_df_oh.tocoo()

In [249]:
pq_df_torch_coo = torch.sparse.LongTensor(torch.LongTensor([pq_df_coo.row.tolist(), pq_df_coo.col.tolist()]),
                              torch.LongTensor(pq_df_coo.data.astype(np.int32)))

In [251]:
model = torch.nn.Sequential()

In [258]:
model.add_module('linear', torch.nn.Linear(in_features=pq_df_torch_coo.shape[1], out_features=1))

In [262]:
torch.optim.SGD(model.parameters(), lr=0.0001)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.0001
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [267]:
loss = torch.nn.Sigmoid()

In [268]:
res_torch_train = torch.tensor(pq_df['res'])

KeyboardInterrupt: 

In [ ]:
for i in range(1000):
    